### Load tensorflow

In [85]:
import tensorflow as tf

print(tf.__version__)

1.14.0


### Enable Eager Execution if you are using tensorflow 1.x

In [86]:
tf.enable_eager_execution()

### Collect Data

In [87]:
import pandas as pd

In [88]:
data = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [89]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [90]:
revised_data = data.drop(columns=['date', 'symbol'])

In [91]:
revised_data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [92]:
dataset = revised_data.head(1000)
dataset.shape

(1000, 5)

In [93]:
dataset.dtypes

open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

### Convert Float64 to Float32

In [94]:
dataset = dataset.astype('float32')
dataset.dtypes

open      float32
close     float32
low       float32
high      float32
volume    float32
dtype: object

### Divide the data into train and test sets

In [95]:
dataset.head()

from sklearn.model_selection import train_test_split

y = dataset['close']
X = dataset.drop(columns=['close'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .30, random_state=1)


In [96]:
import numpy as np
y_train=np.array(y_train)

### Normalize Train and Test Data 

In [97]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)

## Building the graph in tensorflow

2.Define Weights and Bias

In [98]:
# Initialize the weights and bias
X_train.shape

w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

print(w)
print(b)

tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]], shape=(4, 1), dtype=float32)
tf.Tensor([0.], shape=(1,), dtype=float32)


3.Prediction

In [99]:
def prediction(x, w, b):
    # Apply the formula y = wx + b
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [100]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [101]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:        
        t.watch([w,b])    
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [102]:
# I keep getting the Kernel died error. Hence the temporary fix for that is the below statement.
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [103]:
for i in range(100):    
    w, b = train(X_train, y_train, w, b)
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train, w, b)).numpy())

Current Loss on iteration 0 7653.4473
Current Loss on iteration 1 7333.0874
Current Loss on iteration 2 7037.8447
Current Loss on iteration 3 6765.7476
Current Loss on iteration 4 6514.9805
Current Loss on iteration 5 6283.879
Current Loss on iteration 6 6070.889
Current Loss on iteration 7 5874.6055
Current Loss on iteration 8 5693.7046
Current Loss on iteration 9 5526.99
Current Loss on iteration 10 5373.3394
Current Loss on iteration 11 5231.74
Current Loss on iteration 12 5101.24
Current Loss on iteration 13 4980.973
Current Loss on iteration 14 4870.1343
Current Loss on iteration 15 4767.986
Current Loss on iteration 16 4673.8433
Current Loss on iteration 17 4587.082
Current Loss on iteration 18 4507.1235
Current Loss on iteration 19 4433.439
Current Loss on iteration 20 4365.525
Current Loss on iteration 21 4302.9395
Current Loss on iteration 22 4245.2573
Current Loss on iteration 23 4192.0996
Current Loss on iteration 24 4143.1055
Current Loss on iteration 25 4097.9575
Current L

### Get the shapes and values of W and b

In [106]:
print('Weight Shape:\n', w.numpy().shape)
print('Weight Values:\n', w.numpy())

Weights:
 (4, 1)
Weights:
 [[2.6111761e-03]
 [2.5876048e-03]
 [2.6312114e-03]
 [3.2731915e+01]]


In [108]:
print('Bias Shape:\n',b.numpy().shape)
print('Bias Values:\n',b.numpy())

Bias Shape:
 (1,)
Bias Values:
 [32.73192]


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [110]:
# Initialize Sequential Graph (model)
model = tf.keras.Sequential()

# Add Dense layer of 1 for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

# Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [112]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
700/700 [==============================] - 0s 290us/sample - loss: 5637.2866
Epoch 2/100
700/700 [==============================] - 0s 29us/sample - loss: 3907.6253
Epoch 3/100
700/700 [==============================] - 0s 29us/sample - loss: 3627.7416
Epoch 4/100
700/700 [==============================] - 0s 27us/sample - loss: 3580.6417
Epoch 5/100
700/700 [==============================] - 0s 30us/sample - loss: 3571.3847
Epoch 6/100
700/700 [==============================] - 0s 34us/sample - loss: 3571.4402
Epoch 7/100
700/700 [==============================] - 0s 31us/sample - loss: 3572.0100
Epoch 8/100
700/700 [==============================] - 0s 31us/sample - loss: 3569.9313
Epoch 9/100
700/700 [==============================] - 0s 33us/sample - loss: 3569.8464
Epoch 10/100
700/700 [==============================] - 0s 32us/sample - loss: 3572.4515
Epoch 11/100
700/700 [==============================] - 0s 34us/sample - loss: 3570.7857
Epoch 12/100
700/700 [=======

700/700 [==============================] - 0s 26us/sample - loss: 3570.0634
Epoch 94/100
700/700 [==============================] - 0s 28us/sample - loss: 3570.4512
Epoch 95/100
700/700 [==============================] - 0s 29us/sample - loss: 3570.5727
Epoch 96/100
700/700 [==============================] - 0s 27us/sample - loss: 3569.7248
Epoch 97/100
700/700 [==============================] - 0s 27us/sample - loss: 3571.8965
Epoch 98/100
700/700 [==============================] - 0s 29us/sample - loss: 3571.6776
Epoch 99/100
700/700 [==============================] - 0s 29us/sample - loss: 3571.5203
Epoch 100/100
700/700 [==============================] - 0s 27us/sample - loss: 3571.9366


### Classification using Keras 

In [114]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [115]:
model.weights

[<tf.Variable 'dense/kernel:0' shape=(4, 1) dtype=float32, numpy=
 array([[ 0.23466665],
        [ 1.2711728 ],
        [ 1.2771614 ],
        [34.155354  ]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([33.313766], dtype=float32)>]

### Load the given Iris data using pandas (Iris.csv)

In [116]:
iris_data = pd.read_csv('./Iris.csv')
iris_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [172]:
X_i = iris_data.drop(columns=['Species', 'Id'])
y_i = iris_data['Species']
# X_i.shape
y_i

0         Iris-setosa
1         Iris-setosa
2         Iris-setosa
3         Iris-setosa
4         Iris-setosa
5         Iris-setosa
6         Iris-setosa
7         Iris-setosa
8         Iris-setosa
9         Iris-setosa
10        Iris-setosa
11        Iris-setosa
12        Iris-setosa
13        Iris-setosa
14        Iris-setosa
15        Iris-setosa
16        Iris-setosa
17        Iris-setosa
18        Iris-setosa
19        Iris-setosa
20        Iris-setosa
21        Iris-setosa
22        Iris-setosa
23        Iris-setosa
24        Iris-setosa
25        Iris-setosa
26        Iris-setosa
27        Iris-setosa
28        Iris-setosa
29        Iris-setosa
            ...      
120    Iris-virginica
121    Iris-virginica
122    Iris-virginica
123    Iris-virginica
124    Iris-virginica
125    Iris-virginica
126    Iris-virginica
127    Iris-virginica
128    Iris-virginica
129    Iris-virginica
130    Iris-virginica
131    Iris-virginica
132    Iris-virginica
133    Iris-virginica
134    Iri

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [173]:
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

y_i = LabelEncoder().fit_transform(y_i)
# y_i = y_i.reshape(len(y_i), )
print(y_i)

# y_i_one_hot = pd.get_dummies(y_i, sparse=True)
# print(y_i_one_hot)


# binary encode
# onehot_encoder = OneHotEncoder(sparse=False)
# y_i_integer_encoded = y_i_integer_encoded.reshape(len(y_i_integer_encoded), 1)
# print(y_i_integer_encoded)
# onehot_encoded = onehot_encoder.fit_transform(y_i_integer_encoded)
# print(onehot_encoded)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


### Divide the dataset into Training and test (70:30)

In [174]:
X_i_train, X_i_test, y_i_train, y_i_test = train_test_split(X_i, y_i, test_size= .30, random_state=1)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [175]:
# Initialize Sequential model
model1 = tf.keras.models.Sequential()

# Add 1st hidden layer
model1.add(tf.keras.layers.Dense(10, input_shape=(4,)))

# Add 2nd hidden layer
model1.add(tf.keras.layers.Dense(8, input_shape=(4,)))

# Add Output layer
model1.add(tf.keras.layers.Dense(3, activation='softmax'))

# Compile the model
model1.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [176]:
model1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 10)                50        
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 27        
Total params: 165
Trainable params: 165
Non-trainable params: 0
_________________________________________________________________


### Fitting the model and predicting 

In [179]:
model1.fit(X_i_train, tf.keras.utils.to_categorical(y_i_train),          
          validation_data=(X_i_test, tf.keras.utils.to_categorical(y_i_test)),
          epochs=100,
          batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 106us/sample - loss: 0.8531 - acc: 0.6095 - val_loss: 0.7844 - val_acc: 0.6889
Epoch 2/100
105/105 [==============================] - 0s 94us/sample - loss: 0.8320 - acc: 0.5810 - val_loss: 0.7927 - val_acc: 0.5333
Epoch 3/100
105/105 [==============================] - 0s 99us/sample - loss: 0.8050 - acc: 0.5048 - val_loss: 0.7833 - val_acc: 0.6000
Epoch 4/100
105/105 [==============================] - 0s 105us/sample - loss: 0.7700 - acc: 0.6095 - val_loss: 0.9323 - val_acc: 0.6000
Epoch 5/100
105/105 [==============================] - 0s 107us/sample - loss: 0.8101 - acc: 0.6857 - val_loss: 0.7162 - val_acc: 0.6444
Epoch 6/100
105/105 [==============================] - 0s 119us/sample - loss: 0.7327 - acc: 0.6476 - val_loss: 0.7259 - val_acc: 0.6000
Epoch 7/100
105/105 [==============================] - 0s 115us/sample - loss: 0.7142 - acc: 0.6381 - val_loss: 0.8102 - val_acc: 0.600

105/105 [==============================] - 0s 105us/sample - loss: 0.3497 - acc: 0.9619 - val_loss: 0.3962 - val_acc: 0.8444
Epoch 61/100
105/105 [==============================] - 0s 103us/sample - loss: 0.3438 - acc: 0.9429 - val_loss: 0.3736 - val_acc: 0.9333
Epoch 62/100
105/105 [==============================] - 0s 94us/sample - loss: 0.3446 - acc: 0.9619 - val_loss: 0.4085 - val_acc: 0.7556
Epoch 63/100
105/105 [==============================] - 0s 102us/sample - loss: 0.3415 - acc: 0.9333 - val_loss: 0.4531 - val_acc: 0.6222
Epoch 64/100
105/105 [==============================] - 0s 106us/sample - loss: 0.3919 - acc: 0.7619 - val_loss: 0.4256 - val_acc: 0.6889
Epoch 65/100
105/105 [==============================] - 0s 100us/sample - loss: 0.3496 - acc: 0.9048 - val_loss: 0.4947 - val_acc: 0.6222
Epoch 66/100
105/105 [==============================] - 0s 94us/sample - loss: 0.3308 - acc: 0.8571 - val_loss: 0.3978 - val_acc: 0.7333
Epoch 67/100
105/105 [===========================

In [180]:
pred = model1.predict(X_i_test[15:20])
pred

array([[9.7156602e-01, 2.8358692e-02, 7.5313874e-05],
       [2.0573163e-02, 7.0982122e-01, 2.6960564e-01],
       [5.9334859e-03, 4.4647983e-01, 5.4758674e-01],
       [9.8049498e-01, 1.9483736e-02, 2.1302709e-05],
       [9.8438615e-01, 1.5598268e-02, 1.5655754e-05]], dtype=float32)

### Report Accuracy of the predicted values

In [182]:
X_i_test[15:20]

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
42,4.4,3.2,1.3,0.2
92,5.8,2.6,4.0,1.2
66,5.6,3.0,4.5,1.5
31,5.4,3.4,1.5,0.4
35,5.0,3.2,1.2,0.2


In [184]:
y_i_test[15:20]

array([0, 1, 1, 0, 0])